In [1]:
import os
import pandas as pd

df_total =  []
for file_name in os.listdir('./data/product'):
    df_total.append(pd.read_csv(os.path.join('./data/product', file_name)))


df_last = pd.concat(df_total, axis = 0)

In [2]:
df_last.shape

(58, 7)

In [3]:
df_last = df_last.dropna()

In [4]:
df_last.head()

,url,item_name,colors,prices,origin_price,renew_value,technical_infomation
0,https://hoanghamobile.com/dien-thoai/iphone-16...,iPhone 16 Pro Max (256GB) - Chính hãng VN/A,"Titan Tự Nhiên,Titan Sa Mạc,Titan Đen,Titan Trắng","32,490,000 ₫,32,490,000 ₫,32,490,000 ₫,32,490,...","33,690,000 ₫","30,490,000 ₫",<Màn hình>|Tần số quét (Hz): 10-120Hz|Công ngh...
1,https://hoanghamobile.com/dien-thoai/iphone-16,iPhone 16 (128GB) - Chính hãng VN/A,"Đen,Hồng,Xanh Mòng Két,Xanh Lưu Ly,Trắng","20,990,000 ₫,20,990,000 ₫,20,990,000 ₫,20,990,...","22,990,000 ₫","18,990,000 ₫",<Màn hình>|Độ sáng màn hình: 2000nits|Kích thư...
2,https://hoanghamobile.com/dien-thoai/iphone-16...,Điện thoại iPhone 16 Pro - Chính hãng VN/A,"Titan Tự Nhiên,Titan Sa Mạc,Titan Đen,Titan Trắng","26,990,000 ₫,26,990,000 ₫,26,990,000 ₫,26,990,...","26,990,000 ₫","24,990,000 ₫",<Màn hình>|Công nghệ màn hình: Super Retina XD...
3,https://hoanghamobile.com/dien-thoai/iphone-16...,iPhone 16 Plus (128GB) - Chính hãng VN/A,"Đen,Hồng,Xanh Mòng Két,Xanh Lưu Ly,Trắng","24,690,000 ₫,24,690,000 ₫,24,690,000 ₫,24,690,...","24,690,000 ₫","22,690,000 ₫",<Màn hình>|Độ phân giải: 2796 x 1290|Kích thướ...
4,https://hoanghamobile.com/dien-thoai/iphone-16...,iPhone 16 Pro Max (512GB) - Chính hãng VN/A,"Titan Tự Nhiên,Titan Sa Mạc,Titan Đen,Titan Trắng","38,890,000 ₫,38,890,000 ₫,38,890,000 ₫,38,890,...","38,890,000 ₫","36,890,000 ₫",<Màn hình>|Tần số quét (Hz): 10-120Hz|Công ngh...


In [5]:
data = df_last[df_last['item_name'] == 'Samsung Galaxy A06 4GB/128GB']['technical_infomation'][6]

In [6]:
def process_text_dict(dictionary):
    text_processed ={}
    for key, value in dictionary.items():
        if type(value) == str:
            text_processed[key] = value.replace('\n', '.').replace('•', '').strip()
        else:
            text_processed[key] = value
    return text_processed

# Xử lí từng thông số 

In [7]:
import re
def process_infomation(pattern, data):
    try:
        data = re.search(pattern, data, re.S)
        content = data.group(1).strip()
        field_pattern = r"([\w\s()]+): ([^\|]+)"
        fields = re.findall(field_pattern, content)
        info = {key.strip(): value.strip() for key, value in fields}
        return info, content
    except Exception as e:
        print(f'Không tìm thấy {pattern} trong chuỗi')
    

# Xử lí thông số màn hình

In [8]:
import math
def process_screen_data(data):
    screen_pattern = r"<Màn hình>(.*?)<Màn hình>"
    screen, content = process_infomation(screen_pattern, data)
    content = content[1:-1].replace('\n', ',').replace('|', '.').replace('•', '')
    # Xử lí tần số quét (Hz): lấy giá trị tần số quét lớn nhất 
    numbers = re.findall(r"\d+", screen['Tần số quét (Hz)'])
    fre = max(map(int, numbers))
    # Xử lí độ phân giải màn hình: Nhân 2 số với nhau
    numbers = re.findall(r"\d+", screen['Độ phân giải'])
    if len(numbers) == 2:
        resolution = [math.prod(map(int, numbers)), 0]
    elif len(numbers) == 4:
        resolution = [math.prod(list(map(int, numbers[0:2]))), math.prod(list(map(int, numbers[2:4])))]
    # Xử lí kích thước màn hình: Với các dòng Samsung sẽ có cả màn hình phụ 
    numbers = re.findall(r"-?\d+\.\d+", screen['Kích thước màn hình'])
    if len(numbers) == 1:
        size = [float(numbers[0]), 0]
    elif len(numbers) == 2:
        size = list(map(float, numbers))
    # Xử lí độ sáng màn hình
    numbers = re.findall(r"\d+", screen['Độ sáng màn hình'])
    brightness = int(numbers[0])
    screen_info = {
        'Tần số quét (Hz)': fre,
        'Công nghệ màn hình': screen['Công nghệ màn hình'],
        'Độ phân giải màn hình chính' : resolution[0],
        'Độ phân giải màn hình phụ': resolution[1],
        'Kích thước màn hình chính': size[0],
        'Kích thước màn hình phụ': size[1],
        'Độ sáng màn hình': brightness
    }

    return screen_info, content
    

    

info,content = process_screen_data(data)


In [9]:
content


'Tần số quét (Hz): 60Hz.Kích thước màn hình: 6.7".Công nghệ màn hình: PLS LCD.Độ phân giải: 720 x 1600.Độ sáng màn hình: 0'

# Xử lí Camera sau và Camera trước

In [10]:
def process_front_camera(data):
    front_camera_pattern = r"<Camera trước>(.*?)<Camera trước>"
    front_camera, content = process_infomation(front_camera_pattern, data)
    content = content[1:-1].replace('\n', ',').replace('|', '.').replace('•', '')
    front_camera  = process_text_dict(front_camera)
    return front_camera,content


def process_behind_camera(data):
    behind_camera_pattern = r"<Camera sau>(.*?)<Camera sau>"
    behind_camera, content = process_infomation(behind_camera_pattern, data)
    content = content[1:-1].replace('\n', ',').replace('|', '.').replace('•', '')
    behind_camera  = process_text_dict(behind_camera)
    return behind_camera, content
info, cam_content = process_front_camera(data)

In [11]:
cam_content

'Độ phân giải camera: 8 MP'

# Hệ điều hành & CPU

In [12]:
def process_oscpu(data):
    oscpu_pattern = r"<Hệ điều hành & CPU>(.*?)<Hệ điều hành & CPU>"
    oscpu, content = process_infomation(oscpu_pattern, data)
    content = content[1:-1].replace('\n', ',').replace('|', '.').replace('•', '')
    oscpu = process_text_dict(oscpu)
    return oscpu, content

oscpu_info, os_content = process_oscpu(data)

In [13]:
os_content

'Vi xử lý: MediaTek G85.Hệ điều hành: Android.Tốc độ CPU: Octa-core (2x2.0 GHz Cortex-A75 & 6x1.8 GHz Cortex-A55).Vi xử lý đồ họa (GPU): ARM Mali-G52 MC2'

# Bộ nhớ & Lưu trữ

In [14]:
def process_memory(data):
    memory_pattern = r"<Bộ nhớ & Lưu trữ>(.*?)<Bộ nhớ & Lưu trữ>"
    memory, content = process_infomation(memory_pattern, data)
    content = content[1:-1].replace('\n', ',').replace('|', '.').replace('•', '')
    in_memory = memory['Bộ nhớ trong'][:-2]
    ram = memory['RAM'][:-2]
    memory['Bộ nhớ trong'] = int(in_memory)
    memory['RAM'] = int(ram)
    memory = process_text_dict(memory)
    return memory, content
_, content = process_memory(data)

In [15]:
content

'RAM: 4GB.Bộ nhớ trong: 128GB.Thẻ nhớ ngoài: MicroSD (lên đến 1TB)'

# Kết nối

In [16]:
def process_connection(data):
    connection_pattern = r"<Kết nối>(.*?)<Kết nối>"
    connecton, content = process_infomation(connection_pattern, data)
    content = content[1:-1].replace('\n', ',').replace('|', '.').replace('•', '')
    connecton = process_text_dict(connecton)
    return connecton, content


_, content = process_connection(data)

In [17]:
content

'Số khe SIM: 2 SIM (Nano-SIM).Jack tai nghe: 3.5mm.Bluetooth: V5.3.Cổng kết nối/sạc: Type-C.Kết nối khác: Đang cập nhật.Wi-Fi: 802.11a/b/g/n/ac.Cảm biến: Cảm biến vân tay ở nút nguồn cạnh bên.Mạng di động: 4G.Định vị: GPS,Glonass,Beidou,Galileo.Kết Nối NFC: Không'

# Thiết kế - trọng lượng

In [18]:
def process_weight(data):
    weight_pattern = r"<Thiết kế - trọng lượng>(.*?)<Thiết kế - trọng lượng>"
    weight, content = process_infomation(weight_pattern, data)
    content = content[1:-1].replace('\n', ',').replace('|', '.').replace('•', '')
    weight['Trọng lượng'] = int(weight['Trọng lượng'][:-1])
    weight = process_text_dict(weight)
    return weight, content

# Xử lý Pin & Sạc

In [19]:
def process_battery(data):
    battery_pattern = r"<Pin & Sạc>(.*?)<Pin & Sạc>" 
    battery, content = process_infomation(battery_pattern, data)
    content = content[1:-1].replace('\n', ',').replace('|', '.').replace('•', '')
    battery['Hỗ trợ sạc tối đa'] = int(battery['Hỗ trợ sạc tối đa'][:-1])
    if 'Dung lượng pin sản phẩm' not in battery:
        battery['Dung lượng pin sản phẩm'] = 0
    else:
        battery['Dung lượng pin sản phẩm'] = int(battery['Dung lượng pin sản phẩm'][:-3])
    battery = process_text_dict(battery)
    return battery, content

In [20]:
_, content = process_battery(data)
content

'Hỗ trợ sạc tối đa: 25W.Dung lượng pin sản phẩm: 5000mAh.Loại pin: Li-Po.Công nghệ pin: Sạc nhanh 25W'

# Tiện ích

In [21]:
def process_utils(data):
    utils_pattern = r"<Tiện ích>(.*?)<Tiện ích>" 
    utils, content = process_infomation(utils_pattern, data)
    content = content[1:-1].replace('\n', ',').replace('|', '.').replace('•', '')
    utils = process_text_dict(utils)
    return utils, content

_, content = process_utils(data)
content

'Tính năng đặc biệt: Samsung Knox Vault bảo mật gấp bội, nhờ phím khóa vật lý bảo vệ PINS, mật khẩu và các thông tin cá nhân khác để các tin tặc không thể tấn công.Kháng nước, kháng bụi: Đang cập nhật.Bảo mật sinh trắc học: Mở khoá vân tay cạnh viền,Mở khoá khuôn mặt'

# Thông tin chung

In [22]:
def process_general_info(data):
    general_pattern = r"<Thông tin chung>(.*?)<Thông tin chung>" 
    general_info, content = process_infomation(general_pattern, data)
    content = content[1:-1].replace('\n', ',').replace('|', '.').replace('•', '')
    general_info = process_text_dict(general_info)
    return general_info, content

_, content = process_general_info(data)
content

'Thời điểm ra mắt: 08/2024.Sản phẩm bao gồm: Hộp, Sách hướng dẫn, Cây lấy sim, Cáp Type C'

In [23]:
df_last1 = df_last

In [24]:
def process_row(row):
    try:
        screen, screen_content =  process_screen_data(row['technical_infomation'])
        front_camera, front_camera_content = process_front_camera(row['technical_infomation'])
        behind_camera, behind_camera_content = process_behind_camera(row['technical_infomation'])
        os_cpu, os_cpu_content = process_oscpu(row['technical_infomation'])
        memory, memory_content = process_memory(row['technical_infomation'])
        connection, connection_content = process_connection(row['technical_infomation'])
        weight, weight_content = process_weight(row['technical_infomation'])
        battery, battery_content = process_battery(row['technical_infomation'])
        utils, utils_content = process_utils(row['technical_infomation'])
        general_info, general_info_content = process_general_info(row['technical_infomation'])

        row['screen_freq'] = screen['Tần số quét (Hz)']
        row['screen_tech'] = screen['Công nghệ màn hình']
        # row['screen_resolution#1'] = screen['Độ phân giải màn hình chính']
        # row['screen_resolution#2'] = screen['Độ phân giải màn hình phụ']
        row['screen_size'] = screen['Kích thước màn hình chính']
        # row['screen_size#2'] = screen['Kích thước màn hình phụ']
        # row['screen_brightness'] = screen['Độ sáng màn hình']
        
        row['in_memory'] = memory['Bộ nhớ trong']
        row['RAM'] = memory['RAM']

        # row['charging_support'] = battery['Hỗ trợ sạc tối đa']
        row['battery_capacity'] = battery['Dung lượng pin sản phẩm']

        technical_processed = screen_content + front_camera_content + behind_camera_content + os_cpu_content + memory_content + connection_content + weight_content + battery_content + utils_content + general_info_content
        row['technical_infomation'] = technical_processed
    except Exception as e:
        print(e)        

    return row

In [25]:
df_last1 = df_last1.apply(process_row, axis=1)

In [26]:
df_last1

,url,item_name,colors,prices,origin_price,renew_value,technical_infomation,screen_freq,screen_tech,screen_size,in_memory,RAM,battery_capacity
0,https://hoanghamobile.com/dien-thoai/iphone-16...,iPhone 16 Pro Max (256GB) - Chính hãng VN/A,"Titan Tự Nhiên,Titan Sa Mạc,Titan Đen,Titan Trắng","32,490,000 ₫,32,490,000 ₫,32,490,000 ₫,32,490,...","33,690,000 ₫","30,490,000 ₫",Tần số quét (Hz): 10-120Hz.Công nghệ màn hình:...,120,Super Retina XDR,6.9,256,8,0
1,https://hoanghamobile.com/dien-thoai/iphone-16,iPhone 16 (128GB) - Chính hãng VN/A,"Đen,Hồng,Xanh Mòng Két,Xanh Lưu Ly,Trắng","20,990,000 ₫,20,990,000 ₫,20,990,000 ₫,20,990,...","22,990,000 ₫","18,990,000 ₫",Độ sáng màn hình: 2000nits.Kích thước màn hình...,60,Super Retina XDR,6.1,128,8,0
2,https://hoanghamobile.com/dien-thoai/iphone-16...,Điện thoại iPhone 16 Pro - Chính hãng VN/A,"Titan Tự Nhiên,Titan Sa Mạc,Titan Đen,Titan Trắng","26,990,000 ₫,26,990,000 ₫,26,990,000 ₫,26,990,...","26,990,000 ₫","24,990,000 ₫",Công nghệ màn hình: Super Retina XDR.Độ phân g...,120,Super Retina XDR,6.3,128,8,0
3,https://hoanghamobile.com/dien-thoai/iphone-16...,iPhone 16 Plus (128GB) - Chính hãng VN/A,"Đen,Hồng,Xanh Mòng Két,Xanh Lưu Ly,Trắng","24,690,000 ₫,24,690,000 ₫,24,690,000 ₫,24,690,...","24,690,000 ₫","22,690,000 ₫",Độ phân giải: 2796 x 1290.Kích thước màn hình:...,60,Super Retina XDR,6.7,128,8,0
4,https://hoanghamobile.com/dien-thoai/iphone-16...,iPhone 16 Pro Max (512GB) - Chính hãng VN/A,"Titan Tự Nhiên,Titan Sa Mạc,Titan Đen,Titan Trắng","38,890,000 ₫,38,890,000 ₫,38,890,000 ₫,38,890,...","38,890,000 ₫","36,890,000 ₫",Tần số quét (Hz): 10-120Hz.Công nghệ màn hình:...,120,Super Retina XDR,6.9,512,8,0
5,https://hoanghamobile.com/dien-thoai/iphone-16...,iPhone 16 (256GB) - Chính hãng VN/A,"Đen,Hồng,Xanh Lưu Ly,Trắng,Xanh Mòng Két","23,690,000 ₫,23,690,000 ₫,23,690,000 ₫,23,690,...","23,690,000 ₫","21,690,000 ₫",Công nghệ màn hình: Super Retina XDR.Độ phân g...,60,Super Retina XDR,6.1,256,8,0
6,https://hoanghamobile.com/dien-thoai/iphone-16...,iPhone 16 Pro (256GB) - Chính hãng VN/A,"Titan Tự Nhiên,Titan Sa Mạc,Titan Đen,Titan Trắng","29,990,000 ₫,29,990,000 ₫,29,990,000 ₫,29,990,...","29,990,000 ₫","27,990,000 ₫",Công nghệ màn hình: Super Retina XDR.Độ phân g...,120,Super Retina XDR,6.3,256,8,0
7,https://hoanghamobile.com/dien-thoai/iphone-16...,iPhone 16 Plus (256GB) - Chính hãng VN/A,"Đen,Hồng,Xanh Lưu Ly,Trắng,Xanh Mòng Két","27,390,000 ₫,27,390,000 ₫,27,390,000 ₫,27,390,...","27,390,000 ₫","25,390,000 ₫",Tần số quét (Hz): 60Hz.Công nghệ màn hình: Sup...,60,Super Retina XDR,6.7,128,8,0
8,https://hoanghamobile.com/dien-thoai/iphone-16...,iPhone 16 Pro (512GB) - Chính hãng VN/A,"Titan Tự Nhiên,Titan Sa Mạc,Titan Đen,Titan Trắng","36,190,000 ₫,36,190,000 ₫,36,190,000 ₫,36,190,...","36,190,000 ₫","34,190,000 ₫",Công nghệ màn hình: Super Retina XDR.Độ phân g...,120,Super Retina XDR,6.3,256,8,0
9,https://hoanghamobile.com/dien-thoai/iphone-16...,iPhone 16 Plus (512GB) - Chính hãng VN/A,"Đen,Hồng,Xanh Lưu Ly,Trắng,Xanh Mòng Két","31,590,000 ₫,31,590,000 ₫,31,590,000 ₫,31,590,...","31,590,000 ₫","29,590,000 ₫",Tần số quét (Hz): 60Hz.Công nghệ màn hình: Sup...,60,Super Retina XDR,6.7,512,8,0


In [27]:
df_last1.to_csv('./data/final.csv', index = False)

In [28]:
import pandas as pd
df = pd.read_csv('data/final.csv')
df.head()

,url,item_name,colors,prices,origin_price,renew_value,technical_infomation,screen_freq,screen_tech,screen_size,in_memory,RAM,battery_capacity
0,https://hoanghamobile.com/dien-thoai/iphone-16...,iPhone 16 Pro Max (256GB) - Chính hãng VN/A,"Titan Tự Nhiên,Titan Sa Mạc,Titan Đen,Titan Trắng","32,490,000 ₫,32,490,000 ₫,32,490,000 ₫,32,490,...","33,690,000 ₫","30,490,000 ₫",Tần số quét (Hz): 10-120Hz.Công nghệ màn hình:...,120,Super Retina XDR,6.9,256,8,0
1,https://hoanghamobile.com/dien-thoai/iphone-16,iPhone 16 (128GB) - Chính hãng VN/A,"Đen,Hồng,Xanh Mòng Két,Xanh Lưu Ly,Trắng","20,990,000 ₫,20,990,000 ₫,20,990,000 ₫,20,990,...","22,990,000 ₫","18,990,000 ₫",Độ sáng màn hình: 2000nits.Kích thước màn hình...,60,Super Retina XDR,6.1,128,8,0
2,https://hoanghamobile.com/dien-thoai/iphone-16...,Điện thoại iPhone 16 Pro - Chính hãng VN/A,"Titan Tự Nhiên,Titan Sa Mạc,Titan Đen,Titan Trắng","26,990,000 ₫,26,990,000 ₫,26,990,000 ₫,26,990,...","26,990,000 ₫","24,990,000 ₫",Công nghệ màn hình: Super Retina XDR.Độ phân g...,120,Super Retina XDR,6.3,128,8,0
3,https://hoanghamobile.com/dien-thoai/iphone-16...,iPhone 16 Plus (128GB) - Chính hãng VN/A,"Đen,Hồng,Xanh Mòng Két,Xanh Lưu Ly,Trắng","24,690,000 ₫,24,690,000 ₫,24,690,000 ₫,24,690,...","24,690,000 ₫","22,690,000 ₫",Độ phân giải: 2796 x 1290.Kích thước màn hình:...,60,Super Retina XDR,6.7,128,8,0
4,https://hoanghamobile.com/dien-thoai/iphone-16...,iPhone 16 Pro Max (512GB) - Chính hãng VN/A,"Titan Tự Nhiên,Titan Sa Mạc,Titan Đen,Titan Trắng","38,890,000 ₫,38,890,000 ₫,38,890,000 ₫,38,890,...","38,890,000 ₫","36,890,000 ₫",Tần số quét (Hz): 10-120Hz.Công nghệ màn hình:...,120,Super Retina XDR,6.9,512,8,0


In [29]:
for index, row in df.iterrows():
    df.at[index, 'origin_price'] = int(row['origin_price'].replace('₫', '').replace(',', ''))
    df.at[index, 'renew_value'] = int(row['renew_value'].replace('₫', '').replace(',', ''))
    colors_list = row['colors'].split(',')
    prices_list = re.findall(r'\d{1,3}(?:,\d{3})* ₫', row['prices'])
    color_price = ''
    for i in range(len(colors_list)):
        color_price += f"{colors_list[i]} : {prices_list[i]}"
    df.at[index, 'prices'] = color_price

In [30]:
df.head()

,url,item_name,colors,prices,origin_price,renew_value,technical_infomation,screen_freq,screen_tech,screen_size,in_memory,RAM,battery_capacity
0,https://hoanghamobile.com/dien-thoai/iphone-16...,iPhone 16 Pro Max (256GB) - Chính hãng VN/A,"Titan Tự Nhiên,Titan Sa Mạc,Titan Đen,Titan Trắng","Titan Tự Nhiên : 32,490,000 ₫Titan Sa Mạc : 32...",33690000,30490000,Tần số quét (Hz): 10-120Hz.Công nghệ màn hình:...,120,Super Retina XDR,6.9,256,8,0
1,https://hoanghamobile.com/dien-thoai/iphone-16,iPhone 16 (128GB) - Chính hãng VN/A,"Đen,Hồng,Xanh Mòng Két,Xanh Lưu Ly,Trắng","Đen : 20,990,000 ₫Hồng : 20,990,000 ₫Xanh Mòng...",22990000,18990000,Độ sáng màn hình: 2000nits.Kích thước màn hình...,60,Super Retina XDR,6.1,128,8,0
2,https://hoanghamobile.com/dien-thoai/iphone-16...,Điện thoại iPhone 16 Pro - Chính hãng VN/A,"Titan Tự Nhiên,Titan Sa Mạc,Titan Đen,Titan Trắng","Titan Tự Nhiên : 26,990,000 ₫Titan Sa Mạc : 26...",26990000,24990000,Công nghệ màn hình: Super Retina XDR.Độ phân g...,120,Super Retina XDR,6.3,128,8,0
3,https://hoanghamobile.com/dien-thoai/iphone-16...,iPhone 16 Plus (128GB) - Chính hãng VN/A,"Đen,Hồng,Xanh Mòng Két,Xanh Lưu Ly,Trắng","Đen : 24,690,000 ₫Hồng : 24,690,000 ₫Xanh Mòng...",24690000,22690000,Độ phân giải: 2796 x 1290.Kích thước màn hình:...,60,Super Retina XDR,6.7,128,8,0
4,https://hoanghamobile.com/dien-thoai/iphone-16...,iPhone 16 Pro Max (512GB) - Chính hãng VN/A,"Titan Tự Nhiên,Titan Sa Mạc,Titan Đen,Titan Trắng","Titan Tự Nhiên : 38,890,000 ₫Titan Sa Mạc : 38...",38890000,36890000,Tần số quét (Hz): 10-120Hz.Công nghệ màn hình:...,120,Super Retina XDR,6.9,512,8,0


In [ ]:
df.to_csv('data/final.csv', index = False)